In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os as so
import pathlib as pl
import matplotlib.pyplot as pt
from math import ceil
from itertools import islice
from torch.nn.functional import one_hot
from torchvision.utils import make_grid as grid

In [3]:
# Add local package in path of notebook
so.chdir('..')
package = str(pl.Path(so.getcwd(), '..'))
[lambda: sys.path.append(package), lambda: None][package not in sys.path]()

# Local package imports
from gic import *
from gic.learning.wrappers import ClassifierModule
from gic.learning.focalnet.wrappers import FocalNetClassifier, FocalNetClassifierArgs
from gic.learning.ensemble import BaggingEnsemble, FocalNetEnsemble
from gic.data.dataset import GICDataset
from gic.data.dataloader import GICDataModule

Seed set to 10052


In [4]:
data = GICDataModule(DATA_PATH, 32, 'joint')
ensemble = FocalNetEnsemble(n_models=5, args={
    'lr': 4e-4,
    'groups': 8,
    'norm_layer': 'batch',
    'drop_type': 'spatial',
    'num_classes': GICDataset.num_classes,
    'augment': True,
    'augment_n': 1,
    'augment_m': 11,
    'repeat': 3,
    'dropout': 0.15,
    'layers': 2,
    'chan': 128,
    'weight_decay': 8e-3,
    'dense': 224,
    'reduce': "max",
    'dropout_dense': 0.30,
    'activ_fn': 'LeakyReLU',
    'conv_order': "2 1 0"
})

In [ ]:
ensemble.fit(244, data, False)

In [ ]:
output = ensemble.validate(244, data)

In [5]:
# Create submission file
ensemble.load_ensemble(247)
preds = ensemble.predict(data, 'test').sum(dim=1).argmax(dim=-1)
test = GICDataset(DATA_PATH, 'test')
data = test.data_
data['Class'] = preds
data.to_csv(SUBMISSION_PATH, index=False)